# Introducción 

Cuando trabajamos en el campo del aprendizaje automático, a menudo necesitamos determinar cuando dos muestras tienen distribuciones iguales o diferentes. Podemos responder a esta pregunta haciendo uso de test estadísticos de significancia que pueden cuantificar la probabilidad de que las muestras tengan la misma distribución.

Si nuestros datos no tienen una distribución normal o conocida, debemos de usar como recurso la versión no paramétrica de los test de significancia. Este tipo de tests operan de forma similar a los paramétricos, pero sobre distribuciones libres, requieriendo en primer lugar que los datos reales de nuestras distribuciones sean transformados a datos de tipo rank antes de que el test pueda ser realizado.

# Test estadísticos de significancia no paramétricos

Los test estadísticos no paramétricos son aquellos métodos que no asumen una distribución específica para los datos. Estos test son desarrollados para usarlos con datos ordinales o intervalos, pero en la práctica pueden ser usados en observaciones con datos reales rankeados en lugar de hacer uso de las propias observaciones.

A menudo cuando tenemos dos muestras de datos una pregunta que nos podemos hacer es saber si ambas muestras provienen de una misma población. Esta cuestión puede ser respondida haciendo uso de los test de significancia no paramétricos. La hipótesis nula general de este tipo de test es asumir que ambas muestras son extraídas de una misma población o distribución.

Si después de calcular el test de significancia la hipótesis nula es rechazada, esto indica que existe una evidencia estadística de que las muestras han sido extraídas de diferentes poblaciones, y que la diferencia entre los paramátros estimados para ambas poblaciones (media, mediana), puede ser significativa.

Este tipo de test son utilizados de forma habitual a la hora de realizar comparaciones entre el score extraído por diferentes algoritmos con el fin de demostrar que la diferencia de score entre algoritmos es estadísticamente significativo.

# Test Dataset

A la hora de realizar las pruebas para cada uno de los test estadísticos no paramétricos vamos a proceder a generar el siguiente conjunto de datos.

In [3]:
from numpy.random import seed
from numpy.random import randn

#Fijamos la semilla
seed(1)

#Generamos el conjunto de datos
data1 = 50 + (randn(100)*10)
data2 = 51 + (randn(100)*10)

#Vemos un resumen 
print('data1 :  min:%.3f, max:%.3f' % (min(data1), max(data1)))
print('data2 :  min:%.3f, max:%.3f' % (min(data2), max(data2)))

data1 :  min:26.985, max:71.856
data2 :  min:26.652, max:76.283


# Mann-Whitney U Test

Este nos permite determinar si dos muestras independientes provienen de una misma distribución. El estadístico U se determina de la siguiente forma:
    
$U_{1} = n_{1}*n_{2} + \frac{n_{1}*(n_{1} + 1)}{2} - R_{1}$


$U_{2} = n_{1}*n_{2} + \frac{n_{2}*(n_{2} + 1)}{2} - R_{1}$

Donde tenemos que n1 y n2 son la dimensión de nuestras muestras. Para calcular los valores de R1 y R2 se proceden a seguir los siguientes los pasos:

* Ordenamos los valores de las dos muestras conjuntamente.

* Asignamos un rango de orden a cada valor.

* Se corrigen las ligaduras existentes en los daots. Una ligadura hace referencia a un valor que se repite

Finalmente para obetener los valores de R1 Y R2: 

* R1 -> suma de los rangos de la primera muestra.

* R2 -> suma de los rangos de la segunda muestra.

Ejemplo: supongamos que tenemos dos muestras que nos indican la cantidad en Kg que consumen personas de forma mensual en función de la comunidad autónoma. Nuestro objetivo es ver si de alguna manera es ver si la cantidad de Kg de carne consumidos tienen dependencia de la comunidad autónoma en la cuál resida una persona.

In [4]:
#Muestra de la comunidad autónoma A
A = [16,11,14,21,18,34,22,7,12,12]

#Muestra de la comunidad autónoma B
B = [12,14,11,30,10]

#Ordenamos las muestras conjuntamente
AB_ordenada = [7,10,11,11,12,12,12,14,14,16,18,21,22,30,34]

#Asignamos un orden de rango
AB_rank = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

#Corregimos la ligadura promediando
AB_rank_no_ligaduras = [1,2,3.5,3.5,5.5,5.5,6,6,6,8.5,8.5,10,11,12,13,14,15]

#Calculamos R1 y R2
R1 = 10+3.5+8.5+12+11+15+13+1+5.5+5.5
R2 = 5.5+8.5+3.5+14+2

print('R1: {}'.format(R1))
print('R2: {}'.format(R2))

R1: 85.0
R2: 33.5


Para obtener el valor del estadístico U cogemos el mínimo de U1 y U2 tras hacer los cálculas pertinentes

In [8]:
U1 = len(A)*len(B) + (len(A)*(len(A) + 1))/2 - R1
U2 = len(A)*len(B) + (len(B)*(len(B) + 1))/2 - R2
U = min(U1, U2)
print(U)

20.0


Una vez tenemos el valor de este estadístico se procede a realizar una serie de transformaciones que nos permiten obtener un p-valor. Este test puede ser implementado mediante la función **mannwhitneyu()** de la librería SciPy.

In [13]:
from scipy.stats import mannwhitneyu

stat,p = mannwhitneyu(A,B)
print('p valor :%.3f' % p)

p valor :0.249


En este caso nuestro p_valor supera el valor de 0.05, por lo tanto no podemos afirmar a partir de estos datos que exista diferencia entre la ingesta mediana de carne entre los habitantes de la comunidad A y la comunidad B.

In [15]:
# Veamos ahora el ejemplo con el conjunto de datos creado 
stat, p = mannwhitneyu(data1, data2)
print('p valor : %.3f' % p)

p valor : 0.043


En este caso podemos ver que nuestro p_valor es inferior a 0.05, por lo tanto podemos decir que existe una significancia estadśitica los suficientemente fuerte como para decir que las muestras son diferentes.

# Test de Wilcoxon 

Es posible que en determinadas situaciones nuestras muestras estén pareadas. Por ejemplo, supongamos dos algoritmos diferentes que han sido evaluados con exactamente el mismo conjunto de train y de test, o el mismo algoritmo pero evaluado con diferentes conjuntos de train. En estos casos las muestras no son independientes, por lo tanto el test de U Mann-Whitney no puede ser usado. Para estos casos podemos hacer uso del test de Wilcoxon, que es la versión del test de Student's.

Este test toma como hipótesis nula que las dos muestras son iguales y por lo tanto como hipótesis alternativa que las dos muestras son difirentes.

Supongamos estamos tratando de ver la efectividad de dos productos para adelgazar. Para esto, lo que vamos hacer es ver como estos dos productos afectan sobre 8 personas, de forma que vamos a ver la cantidad de kilos perdidos por cada paciente en función del producto tomado.

In [4]:
#Ejemplo
import numpy as np 

productoA = np.array([4,3,6,7,9,4,2,5])
productoB = np.array([3,4,6,6,5,4,7,3])

Para realizar el text de Wilcoxon vamos a proceder a realizar los siguientes pasos:

* Calcular las diferencias.

* Eliminar elementos donde la diferencia es cero. Estas muestras no van aportar información para decirnos si un producto se comporta mejor o peor.

* Ordenamos las diferencias prescindiendo de los signos.

* Asignamos rango de orden.

*  Corregimos las ligaduras.

* Sumamos los rangos según los signos que tengan las diferencias y obtenemos los estimadores:

    * T(+) = suma de rangos correspondientes a diferencias positivas
    
    * T(-) = suma de rangos correspondientes a diferencias negativas.
    

* Definimos el estadístico como el T = min(T(+), T(-))

In [10]:
#Hacemos diferencias 
d = productoA - productoB
print(d)

[ 1 -1  0  1  4  0 -5  2]


In [11]:
#Ordenamos la array independiente del signo
d_order = sorted(abs(d))
print(d_order)

[0, 0, 1, 1, 1, 2, 4, 5]


In [13]:
#Eliminar los ceros
d_order_wo0 = [value for value in d_order if value != 0]
print(d_order_wo0)

[1, 1, 1, 2, 4, 5]


In [14]:
#Asignamos rangos de orden 
rank_order = [1,2,3,4,5,6]
print(rank_order)

[1, 2, 3, 4, 5, 6]


In [15]:
#Eliminamos las ligaduras asignando el valor medio
rank_order = [2,2,2,4,5,6]
print(rank_order)

[2, 2, 2, 4, 5, 6]


In [22]:
T_plus = 2 + 2 + 5 + 4 
T_minus = 2 + (6)
print(min(T_minus, T_plus))

8


Tras esto debemos de irnos a una tabla, y de esta forma en función de los valores obtenidos podemos decir si podemos o no rechazar la hipótesis nula.

En python podemos realizar este test a partir de la función **wilcoxon()** que se encuentra en la librería SciPy. 

In [26]:
from scipy.stats import wilcoxon

T, p_value = wilcoxon(productoA, productoB)

print('T : %.3f, p_valor : %.3f' % (T, p_value))

T : 8.000, p_valor : 0.596


/home/francisco/anaconda3/lib/python3.6/site-packages/scipy/stats/morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


Si tomamos un nivel de significancia del 5% (0.05), podemos ver que tenemos un p_valor muy por encima de este valor por lo tanto no tenemos suficiente significancia estadística como para decir que la hipótesis nula es falsa. Es decir, no tenemos suficiente significancia estadística como para admitir que los productos actuan de forma diferente sobre las personas.

# Kruskal-Wallis H Test

Cuando trabajamos con tests de significancia, tales como el test de U Mann-Whitney y Wilcoxon, la comparación entre muestras de datos es por pareja. Esto puede llegar a ser ineficiente si tenemos una gran cantidad de muestras de datos y solo estamos interesados si dos o más muestras tienen una distribución diferente.

El test de Kruskal Wallises una versión no parámetrica del test ANOVA. Este test se puede usar para determinar si más de dos muestras independientes tienen una distribución diferente. La hipótesis nula del test es que todas las muestras fueron extraídas de la misma función de distribución. De forma específica, supone que las medianas de todas las muestras son iguales. Si rechazamos la hipótesis nula, indica que existe suficiente evidencia estadística para sugerir que una o más muestras dominan sobre otra muestra, pero la prueba no indica que muestras ni en que medida.

Para poder aplicar el test se debe cumplir:

* Las muestras deben de ser independientes.

* Cada muestras debe tener como mínimo 5 observaciones.

Además las muestras pueden diferir en su tamaño, es decir, no todas las muestras deben tener el mismo número de observaciones.

In [4]:
from numpy.random import seed, rand
from scipy.stats import kruskal

#Fijamos la semilla
seed(1)

#Generamos 3 muestras independientes 
data1 = 50 + (rand(100)*10)
data2 = 51 + (rand(100)*10)
data3 = 52 + (rand(100)*10)

#Hacemos el test de Kruskal-Wallis
stats, p_valor = kruskal(data1, data2, data3)

print('Estadístico : %.3f, p_valor : %.3f' % (stats, p_valor))

Estadístico : 34.747, p_valor : 0.000


Si fijamos una significancia estadística de 0.05, podemos ver como nuestro p_valor es menor, y por lo tanto podemos decir que existe la significancia estadística suficiente como para decir que al menos dos o más muestras tienen distribuciones diferentes.

# Friedman Test 

Hemos visto como el test de Kruskal Wallis nos permite ver si dos o más nuestras tienen o no la misma función distribución. Si las muestras de alguna manera están pareadas, por ejemplo, medidas repetidas, entonces el test de Kruskal Wallis podría no ser adecuado, en su lugar, podemos hacer uso del test de Friedman.

EL test de Friedman es una versión no paramétrica del análisis de un test de varianza para medidas repetidas. Por defecto, asume que las muestras tienen igual distribución. Si rechazamos la hipótesis nula indicamos que una o más de nuestras muestras pareadas tienen una distribución diferente.

Podemos implementar el test de Friedman mediante la función **friedmanchisquare()** que se encuentra en la librería SciPy.

In [2]:
from numpy.random import seed
from numpy.random import rand
from scipy.stats import friedmanchisquare

#Fijamos la semilla
seed(1)

#Generamos nuestro conjunto de datos 
data1 = 50 + (rand(100) * 10)
data2 = 51 + (rand(100) * 10)
data3 = 52 + (rand(100) * 10)

#Aplicamos el test de Friedman
stat, p = friedmanchisquare(data1, data2, data3)

#Vemos el resultado del p_valor
print("p_valor : %.3f" % p)

p_valor : 0.000


Tomando un nivel de significancia del 5% (0.05), podemos ver como nuestro p_valor es inferior, por lo tanto existe la suficiente significancia estadística para rechazar la hipñótesis nula y por lo tanto una o o ás muestras tienen una distribución diferente.